# IMDB XGBoost Revenue
![ImdbIcon](../images/imdbheader.jpg)

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import linear_model
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV

# Importing so that I can use LinearRegression and OLS Models.
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn import metrics

In [21]:
revenue_df = pd.read_csv('../data/revenue_df.csv')

In [22]:
revenue_df.columns

Index(['movie_title', 'year', 'tagline', 'plot', 'cast', 'duration', 'Action',
       'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Drama',
       'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Musical', 'Mystery',
       'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western', 'avg_vote',
       'total_votes', 'us_voters_votes', 'votes', 'votes_1', 'votes_2',
       'votes_3', 'votes_4', 'votes_5', 'votes_6', 'votes_7', 'votes_8',
       'votes_9', 'votes_10', 'popularity', 'director_score', 'actor_score',
       'actress_score', 'tagline_sentiment', 'plot_sentiment', 'total_score',
       'profitable', 'budget', 'revenue', 'budget_adj', 'revenue_adj'],
      dtype='object')

### XGBoost

In [23]:
from xgboost import XGBRegressor

In [24]:
xgb = XGBRegressor()

In [351]:
features = ['year', 'duration', 'Action', 'Adventure', 'Animation', 'Comedy', 'Drama', 'Family', 'Music', 'Mystery', 
            'Romance', 'Sci-Fi',  'Thriller', 'total_votes', 'us_voters_votes','votes_1', 'votes_2', 'votes_3', 
            'votes_4', 'votes_5', 'votes_6', 'votes_7', 'votes_8', 'votes_9', 'votes_10', 'popularity', 'director_score', 
            'actor_score', 'actress_score', 'tagline_sentiment', 'plot_sentiment', 'total_score', 'profitable']

In [352]:
X = revenue_df[features]
y = revenue_df['revenue']

In [353]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [354]:
xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=24, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [355]:
predictions = xgb.predict(X_train)

residuals = y_train - predictions

rss = (residuals ** 2).sum()

mse = metrics.mean_squared_error(y_train, predictions)

rmse = np.sqrt(metrics.mean_squared_error(y_train, predictions))

In [356]:
print("RMSE:", rmse)
print(" ")
print("Training Score:", xgb.score(X_train, y_train))
print(" ")
print("Testing Score:", xgb.score(X_test, y_test))

RMSE: 3261356.524632305
 
Training Score: 0.9997115123728739
 
Testing Score: 0.6019469423166042


In [357]:
null_predictions = np.zeros_like(y_test)

null_pred = null_predictions + y_test.mean()

baseline = metrics.mean_squared_error(y_test, null_pred, squared = False)

In [358]:
print("Baseline RMSE:", baseline)
print(" ")
print("RMSE:", rmse)
print("________________________________")
print(" ")
print("Difference:", baseline - rmse)

Baseline RMSE: 155540661.6705855
 
RMSE: 3261356.524632305
________________________________
 
Difference: 152279305.1459532


## GridSearch

In [359]:
xgb = XGBRegressor()

In [360]:
model_params = {}
count = 0

In [365]:
xgb_params = {
    'base_score' : [.01, .05],
    'booster' : ['gbtree', 'gblinear', 'dart'],
    'learning_rate': [.10, .15, .20],
    'max_depth' : [1, 2, 3],
    'n_estimators': [150, 175, 200],
}

gs = GridSearchCV(xgb, param_grid = xgb_params, cv = 5, n_jobs = 12)

gs.fit(X_train, y_train)

count += 1

gs.best_params_['best_score'] = gs.best_score_

gs.best_params_['training_score'] = gs.score(X_train, y_train)

gs.best_params_['testing_score'] = gs.score(X_test, y_test)

model_params[f'model_{count}'] = gs.best_params_

model_df = pd.DataFrame.from_dict(model_params, orient = 'index')

model_df

,base_score,booster,learning_rate,max_depth,n_estimators,best_score,training_score,testing_score
model_1,0.05,gbtree,0.10,2,200,0.651482,0.904779,0.548136
model_2,0.01,dart,0.15,2,200,0.654042,0.926342,0.553760
model_3,0.01,dart,0.15,2,200,0.654042,0.926342,0.553760
model_4,0.01,dart,0.15,2,175,0.655008,0.920677,0.552459
model_5,0.01,dart,0.15,2,175,0.655008,0.920677,0.552459
